In [ ]:
import torch
import numpy as np
from PIL import Image
from diffusers import StableDiffusionXLImg2ImgPipeline

In [ ]:
base = StableDiffusionXLImg2ImgPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
).to("cuda", torch.float16)

refiner = StableDiffusionXLImg2ImgPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    text_encoder_2=base.text_encoder_2,
    vae=base.vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16"
    ).to("cuda", torch.float16)

In [ ]:
prompt = "a beautiful cat"
negative_prompt = "lowres, extra digit, fewer digits, cropped, worst quality, low quality, text, word, icon, logo, hands, fingers, feet, face, eyes, anime, women, man, nude"
init_image = Image.open("../1_media/input_images/cat.jpg").convert("RGB")

In [ ]:
seed = np.random.randint(0, np.iinfo(np.int64).max)
generator = torch.Generator().manual_seed(seed)

output = base(image=init_image,
              prompt=prompt,
              negative_prompt=negative_prompt,
              num_inference_steps=50,
              strength=0.5,
              denoising_end=0.8,
              generator=generator,
              guidance_scale=15,
              output_type="latent").images

output = refiner(image=output,
                prompt=prompt,
                negative_prompt=negative_prompt,
                num_inference_steps=50,
                strength=0.5,
                denoising_start=0.8,
                generator=generator,
                guidance_scale=15).images[0]

display(output)